# Libs

In [1]:
from  openai import OpenAI # type: ignore
import minsearch # type: ignore
import json
import textwrap
from elasticsearch import Elasticsearch
# from elasticsearch import NotFoundError
from tqdm.auto import tqdm
import requests 

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# API key

In [25]:
# apikey = open('txt.txt').read()
# client = OpenAI(api_key=apikey)

# Ollama

In [6]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [41]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='ollama-gemma2b',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    return response.choices[0].message.content

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [4]:
prompt = "What is the formula for energy?"

In [3]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [6]:
response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0)

In [29]:
txt = response.choices[0].message.content
textwrap.wrap(txt, width=70)

["Sure, here's the formula for energy:  **E = K + U**  Where:  * **E**",
 'is the energy in joules (J) * **K** is the kinetic energy in joules',
 '(J) * **U** is the potential energy in joules (J)  **Kinetic energy',
 '(K)** is the energy an object possesses when it moves due to its',
 'motion. It is calculated using the formula:  **K = 1/2 * mass *',
 'velocity^2**  Where:  * **mass** is the mass of the object in',
 'kilograms (kg) * **velocity** is the velocity of the object in meters',
 'per second (m/s)  **Potential energy (U)** is the energy an object',
 'possesses due to its position or configuration. It is calculated using',
 'the formula:  **U = m * g * h**  Where:  * **m** is the mass of the',
 'object in kilograms (kg) * **g** is the acceleration due to gravity in',
 'meters per second squared (m/s²) * **h** is the height or vertical',
 'position of the object in meters (m)  **Total energy (E)** is the sum',
 'of the kinetic and potential energies:  **E = K + U**  This formul

In [27]:
response.usage

CompletionUsage(completion_tokens=289, prompt_tokens=33, total_tokens=322)